In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns


In [ ]:
cross_encoder, tokenizer = init_model()
data_provider = InferenceDataProvider()
nr_annotated_passages = 16
nr_annotated_dialogues = 102

df_data = []
for diag_id in range(nr_annotated_dialogues):

    diag_turns, grounded_agent_utterance, nr_show_utterances, rerank_dialog_examples = \
        data_provider.get_dialog_out(dialog_id)
    inf_out = data_provider.get_dialog_inference_out(diag_id, nr_annotated_passages)

    for example in inf_out["reranked_examples"]:
        if "gpt_references" in example:
            passage_tokens, gpt_labels_refs, failed_refs = create_grounding_annt_list(example["passage"],
                                                                                      example["gpt_references"],
                                                                                      1,
                                                                                      return_failed=True)
            
            if not failed_refs:
                passage_list, _ = create_grounding_annt_list(r_example['x'],
                                                                     grounded_agent_utterance["references"],
                                                                     r_example["label"])
                sep_index = [i for i, x in enumerate(passage_list) if x == "[SEP]"][0]
                passage_list = passage_list[sep_index + 1:]
                nr_tokens_trunc_passage = len(passage_list)
                
                # gpt_labels_refs is for whole passage, but inf_out["grad_sam_scores"] is A[SEP]B
                record = {
                    "passage": example["passage"],
                    "passage_tokens": passage_tokens,
                    "gpt-labels-refs": gpt_labels_refs[:nr_tokens_trunc_passage], 
                    "passage-tokens": passage_tokens[:nr_tokens_trunc_passage],
                    "grad-sam-refs": inf_out["grad_sam_scores"][1:][:-1][sep_index + 1:][:nr_tokens_trunc_passage],
                    "passage_w_dialogue": example["x"],
                }
                df_data.append(record)

df = pd.DataFrame(df_data)

df

In [ ]:
# Transforming grad-sam refs to the format of gpt_label_refs and opposite

# Remove all scores before [SEP]
score_colour = score_colour[1:][:-1][:len(passage_list)]
